# Sherpa AI Chatbot

## Imports and NLTK Resource Download
OpenAI, pandas, NumPy, NLTK, and scikit-learn, sentiment analysis, NLTK tokenization resources

In [1]:
import openai
import pandas as pd
import numpy as np
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to /Users/rahulrao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rahulrao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## External Data Preprocessing

In [2]:
# Load and preprocess dataset
data_path = 'Compilation_T1_Master_Preprocessed.csv'  
data = pd.read_csv(data_path)
data['Tags'] = data['Tags'].apply(lambda x: x.lower().split(', '))


## Sentiment Analysis Tooling

In [3]:
def analyze_sentiment(user_input):
    """
    Analyzes the sentiment of a given user input.
    :param user_input: The user input text.
    :return: The sentiment polarity score (-1 to 1).
    """
    return TextBlob(user_input).sentiment.polarity


def find_relevant_advice(user_input):
    """
    Finds relevant advice from the dataset based on user input.
    :param user_input: The user input text.
    :return: Relevant advice text.
    """
    user_input_lower = user_input.lower()
    vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
    summaries = data['Summary'].tolist()
    tfidf_matrix = vectorizer.fit_transform(summaries + [user_input_lower])
    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    most_similar_index = np.argmax(cosine_sim)
    return data.iloc[most_similar_index]['Summary']


## Token Calculation Refinement

In [4]:
def calculate_tokens(text):
    """
    Calculates the number of tokens (words) in a given text.
    :param text: The input text.
    :return: Number of tokens.
    """
    return len(text.split())


def refine_prompt_for_tokens(messages, max_total_tokens=1024):
    """
    Refines the prompt to ensure it doesn't exceed the token limit.
    :param messages: List of messages in the conversation.
    :param max_total_tokens: Maximum total tokens allowed.
    :return: Refined list of messages.
    """
    total_tokens = sum([calculate_tokens(message['content']) for message in messages])
    while total_tokens > max_total_tokens - 150:
        messages.pop(0)
        total_tokens = sum([calculate_tokens(message['content']) for message in messages])
    return messages

## Context Summary
updates response context based on previous inputs by user

In [5]:
def update_context_summary(last_user_message, context_summary):
    """
    Updates the context summary based on the last user message.
    :param last_user_message: The last user message.
    :param context_summary: The current context summary.
    :return: Updated context summary.
    """
    emotional_keywords = {
        "sad": ["sad", "depressed", "unhappy", "sorrow", "down"],
        "anxious": ["anxious", "nervous", "worried", "stressed", "tense"],
        "angry": ["angry", "mad", "furious", "resentful", "irritated"],
        "lonely": ["lonely", "isolated", "alone", "abandoned", "secluded"],
        "confused": ["confused", "uncertain", "lost", "unsure", "doubtful"],
        "hopeless": ["hopeless", "despair", "futile", "bleak", "discouraged"],
        "overwhelmed": ["overwhelmed", "swamped", "burdened", "overloaded", "stretched"],
        "joyful": ["happy", "joyful", "elated", "overjoyed", "ecstatic"],
        "enthusiastic": ["enthusiastic", "excited", "eager", "keen", "animated"],
        "content": ["content", "satisfied", "pleased", "gratified", "fulfilled"],
        "loved": ["loved", "cherished", "valued", "cared for", "appreciated"],
        "motivated": ["motivated", "inspired", "driven", "ambitious", "determined"]
    }

    situational_keywords = {
        "relationship_issues": ["breakup", "divorce", "fight", "argument", "conflict"],
        "work_stress": ["work", "job", "unemployment", "fired", "layoff"],
        "academic_pressure": ["exams", "grades", "school", "homework", "study"],
        "health_concerns": ["sick", "illness", "injury", "health", "hospital"],
        "self_esteem": ["worthless", "inadequate", "inferior", "failure", "insecure"],
        "identity_search": ["who am I", "identity", "myself", "self discovery", "find myself"]
    }

    # Check for emotional context
    for emotion, keywords in emotional_keywords.items():
        if any(keyword in last_user_message for keyword in keywords):
            context_summary["emotion"] = emotion
            break
    
    # Check for situational context
    for concern, keywords in situational_keywords.items():
        if any(keyword in last_user_message for keyword in keywords):
            context_summary["concern"] = concern
            break
    
    return context_summary



## OpenAI Model Creation + Chat Response Function

In [6]:
def get_openai_chat_response(user_input, conversation_history, context_summary, api_key):
    """
    Generates a response from the OpenAI Chat API based on the user input and conversation context.
    :param user_input: The user's input message.
    :param conversation_history: History of the conversation.
    :param context_summary: Summary of the conversation context.
    :param api_key: API key for accessing the OpenAI Chat API.
    :return: Response generated by the AI.
    """
    openai.api_key = api_key
    
    # Dynamically update context summary based on the latest user input
    if conversation_history:
        last_user_message = conversation_history[-1]["content"]
        context_summary = update_context_summary(last_user_message.lower(), context_summary)

    # Start constructing the messages payload for the API call
    messages = [{"role": "system", "content": "You are an empathetic therapist named Sherpa. Provide concise advice responding as a human would"}]
    
    # Include dynamic context messages based on context_summary
    if 'emotion' in context_summary:
        emotion_context_msg = f"The user has expressed feelings of {context_summary['emotion']}."
        messages.append({"role": "system", "content": emotion_context_msg})
    
    if 'concern' in context_summary:
        concern_context_msg = f"The user is dealing with {context_summary['concern']}."
        messages.append({"role": "system", "content": concern_context_msg})
    
    # Add a selection of recent user messages for richer context, if available
    recent_messages = conversation_history[-5:]  # Adjust as needed
    for msg in recent_messages:
        messages.append(msg)

    # Append the latest user input
    messages.append({"role": "user", "content": user_input})

    # Make the API call
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=150  # Adjust based on your needs
        )
        ai_response = response.choices[0].message['content']
        
        # Update conversation history with the latest interaction
        conversation_history.append({"role": "user", "content": user_input})
        conversation_history.append({"role": "assistant", "content": ai_response})
        
        return ai_response
    except Exception as e:
        return f"An error occurred: {str(e)}"


# Main Method

In [ ]:
def main():
    """
    Main function to run the AI therapist chatbot.
    """
    api_key = input("Please enter your OpenAI API key: ")  # Prompt the user for the API key securely
    conversation_history = []
    context_summary = {}  # Initialize an empty context summary for the session

    print("Welcome to PeakMind's Sherpa. How can I assist you today?")

    while True:
        user_input = input("\nYour message: ")
        if user_input.lower() == 'exit':
            print("Sherpa: It's been meaningful to connect with you. Take care.")
            break

        # Call the function to get the AI's response
        ai_response = get_openai_chat_response(user_input, conversation_history, context_summary, api_key)
        
        print("Sherpa:", ai_response)

if __name__ == "__main__":
    main()
